In [1]:
import dev_ga_base

Poblacion creada con 20000 individuos
Inicio del proceso iterativo

###
Paso  0
Puntaje: 4
Fallo: [5, 7, 8, 11, 14]
Mejor:  ['blue', 'Developer', 'JavaScript', 'Cassandra', 'Vim']
Mejor:  ['yellow', 'Analyst', 'C#', 'Neo4j', 'Sublime Text']
Mejor:  ['white', 'Hacker', 'Python', 'MongoDB', 'Notepad++']
Mejor:  ['green', 'Engineer', 'Java', 'HBase', 'Brackets']
Mejor:  ['red', 'Mathematician', 'C++', 'Redis', 'Atom']

###
Paso  1
Puntaje: 6
Fallo: [3, 4, 9, 11]
Mejor:  ['blue', 'Developer', 'C#', 'HBase', 'Brackets']
Mejor:  ['yellow', 'Analyst', 'Python', 'Cassandra', 'Atom']
Mejor:  ['red', 'Mathematician', 'JavaScript', 'Neo4j', 'Sublime Text']
Mejor:  ['white', 'Hacker', 'Java', 'Redis', 'Vim']
Mejor:  ['green', 'Engineer', 'C++', 'MongoDB', 'Notepad++']

###
Paso  2
Puntaje: 8
Fallo: [3, 4, 5]
Mejor:  ['blue', 'Developer', 'Java', 'Redis', 'Brackets']
Mejor:  ['yellow', 'Analyst', 'JavaScript', 'Cassandra', 'Vim']
Mejor:  ['white', 'Hacker', 'C#', 'HBase', 'Notepad++']
Mejor:  ['gre

# Resultado

Tiempo transcurrido **00:00:09.69**

- Fitness: 14 
 
- Individuo ['010', '101', '011', '101', '011', '001', '001', '100', '100', '010', '101', '010', '001', '001', '101', '100', '100', '010', '011', '100', '011', '011', '101', '010', '001'] 

- Individuo [['blue', 'Developer', 'Java', 'Redis', 'Vim'], ['red', 'Mathematician', 'C++', 'Neo4j', 'Sublime Text'], ['yellow', 'Hacker', 'Python', 'Cassandra', 'Notepad++'], ['white', 'Analyst', 'C#', 'HBase', 'Atom'], ['green', 'Engineer', 'JavaScript', 'MongoDB', 'Brackets']]
